# 01b_clean_canada_grocery.ipynb

## Goal
Clean the raw grocery dataset to fix brand inconsistencies and normalize prices.

## Steps
1. Load raw data.
2. Fix brand-product mismatches (e.g., Coke brand on produce).
3. Winsorize prices per category to remove outliers.
4. Export to `outputs/canada_grocery_nutrition_clean.csv`.

In [ ]:
import pandas as pd
from pathlib import Path

raw_path = Path("raw/canada_grocery_nutrition_5000.csv")
df = pd.read_csv(raw_path)

df.head()

In [ ]:
df = df.copy()

# Big packaged-goods brands that shouldn't be on lettuce, cucumbers, etc.
big_cp_brands = {
    "Coca-Cola", "PepsiCo", "Nestlé", "Heinz",
    "Yoplait", "Oikos", "Quaker", "Kellogg's", "Astro"
}

# Map keywords in the product name → correct beverage brand
beverage_keywords = {
    "coke": "Coca-Cola",
    "cola": "Coca-Cola",
    "coca-cola": "Coca-Cola",
    "pepsi": "PepsiCo",
    "sprite": "Coca-Cola",
    "fanta": "Coca-Cola",
    "7-up": "PepsiCo",
    "7up": "PepsiCo",
    "7 up": "PepsiCo",
    "mountain dew": "PepsiCo",
}

produce_categories = {"Produce"}

def clean_brand(row):
    name = str(row["product_name"]).lower()
    brand = str(row["brand"])
    store = str(row["store"])
    category = str(row["category"])

    # 1) If the name clearly says Coke / Pepsi / Sprite etc., force the right brand
    for kw, br in beverage_keywords.items():
        if kw in name:
            return br

    # 2) For Produce, if the brand is a big packaged-goods brand, use the store as brand
    if category in produce_categories and brand in big_cp_brands:
        return store

    # 3) Otherwise keep the original brand
    return brand

df["brand_clean"] = df.apply(clean_brand, axis=1)

df[["product_name", "category", "store", "brand", "brand_clean"]].head(20)

In [ ]:
def winsorize_prices(group, lower=0.01, upper=0.99):
    q_low = group["price_per_100g"].quantile(lower)
    q_high = group["price_per_100g"].quantile(upper)

    # Clamp
    group["price_per_100g_clean"] = group["price_per_100g"].clip(q_low, q_high)

    # Scale other price columns proportionally
    # Avoid division by zero
    mask = group["price_per_100g"] != 0
    factor = pd.Series(1.0, index=group.index)
    factor[mask] = group.loc[mask, "price_per_100g_clean"] / group.loc[mask, "price_per_100g"]
    
    group["price_per_gram_clean"] = group["price_per_gram"] * factor
    group["price_per_serving_clean"] = group["price_per_serving"] * factor

    return group

df = df.groupby("category", group_keys=False).apply(winsorize_prices)

df[["category", "price_per_100g", "price_per_100g_clean"]].head(20)

In [ ]:
# Drop old columns and rename clean ones
df = df.drop(columns=["brand"])
df = df.rename(columns={"brand_clean": "brand"})

df = df.drop(
    columns=["price_per_gram", "price_per_serving", "price_per_100g"]
).rename(
    columns={
        "price_per_gram_clean": "price_per_gram",
        "price_per_serving_clean": "price_per_serving",
        "price_per_100g_clean": "price_per_100g",
    }
)

# Optional: sanity check again
df[["product_name", "brand", "category", "price_per_100g"]].head(20)

In [ ]:
output_path = Path("outputs/canada_grocery_nutrition_clean.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)

print(f"Saved to {output_path}")